# Elements of a prompt

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

chat_model = ChatOpenAI()

conversation_chain = ConversationChain(
    llm=chat_model
)

print(conversation_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


# Few-shot learning

In [2]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "5+6", "output": "11"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())


Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11


In [3]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are wonderous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input='What is 3+3?'))

System: You are wonderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?


In [20]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
    verbose=True
)

chain.run("What is 3+3?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are wonderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?

> Finished chain.


'3+3 is equal to 6.'

# Memetic Proxy

In [21]:
from langchain.prompts import PromptTemplate

template = """
System: {reference}
Provide an helpful answer to the following question

Human: {question}

AI:"""

prompt = PromptTemplate.from_template(template)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

high_level = "Imagine you are a Professor teaching at the PhD level."
lower_level = "Imagine you are a Kindergarten teacher."


question = "What is Quantum Mechanics"

chain.run({
    'question': question,
    'reference': high_level  
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Professor teaching at the PhD level.
Provide an helpful answer to the following question

Human: What is Quantum Mechanics

AI:

> Finished chain.


'Quantum mechanics is a branch of physics that deals with the behavior of matter and energy on the smallest scales, such as atoms and subatomic particles. It provides a mathematical framework to describe the dual nature of particles, which can exhibit both particle-like and wave-like properties. Quantum mechanics is responsible for many counterintuitive phenomena, such as particle-wave duality, superposition, and entanglement. It also provides the foundation for understanding various technological advancements, including transistors, lasers, and quantum computers. Overall, quantum mechanics is a fundamental theory that allows us to understand and explain the behavior of matter and energy at the quantum level.'

In [22]:
chain.run({
    'question': question,
    'reference': lower_level  
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Kindergarten teacher.
Provide an helpful answer to the following question

Human: What is Quantum Mechanics

AI:

> Finished chain.


'Quantum Mechanics is a branch of physics that deals with the behavior of matter and energy on the smallest scales, such as atoms and subatomic particles. It explores the fundamental principles that govern the behavior of these particles, including concepts like superposition, entanglement, and wave-particle duality. While it may seem complex, understanding the principles of Quantum Mechanics can lead to new technological advancements and a deeper appreciation of the nature of the universe.'

# Chain of Thoughts

In [23]:
import pandas as pd

file_path = '/Users/damienbenveniste/Projects/Teaching/Introduction_Langchain/data/ai2_arithmetic_questions.csv'

cot_df = pd.read_csv(file_path)
 
cot_df

,Question,Rationale,Response
0,Blake filled a bucket with 0.8 gallon of water...,Since Blake filled the bucket with 0.8 gallon ...,0.6
1,There is 0.2 cup of oil in Scarlett's measurin...,0.2 + 0.3 = 0.5,0.5
2,Craig walked 0.2 mile from school to David's h...,Craig walked 0.2 mile from school to David's h...,0.9
3,Hannah's Vegetarian Restaurant bought 0.3 poun...,Hannah's Vegetarian Restaurant bought 0.3 poun...,0.9
4,Alyssa bought some toys. She bought a football...,Alyssa spent $5.71 on football and $6.59 on ma...,12.3
...,...,...,...
575,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...","Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",0.3
576,"Mike found 6 seashells and 4 starfish, but 4 o...",The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,2.0
577,Mary loves eating fruits. Mary paid $11.08 for...,Mary loves eating fruits. Mary paid $11.08 for...,34.7
578,Joan has 40 blue balloons Melanie has 41 blue ...,Joan has 40 blue balloons Melanie has 41 blue ...,81.0


In [29]:
list(cot_df.T.to_dict().values())[0]


{'Question': 'Blake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?',
 'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon',
 'Response': 0.6}

In [30]:
import random

random.sample(list(cot_df.T.to_dict().values()), 10)

[{'Question': 'Brennan had 0.25 grams of pepper. Then he used 0.15 grams of thepepper to make some scrambled eggs. How much pepper doesBrennan have?',
  'Rationale': 'Brennan had 0.25 grams of pepper. Then he used 0.15 grams of thepepper to make some scrambled eggs, which means he only has0.25 - 0.15 = 0.1 grams left.""',
  'Response': 0.1},
 {'Question': "At the beach, Miki and her sister both built sandcastles and then measured their heights. Miki's sandcastle was 0.83 foot tall and her sister's was 0.53 foot tall. How much taller was Miki's sandcastle than her sister's?",
  'Rationale': "Miki's sandcastle was 0.83 foot tall and her sister's was 0.53 foot tall. So, Miki's sandcastle was (0.83 - 0.53) = 0.3 foot taller than her sister's sandcastle",
  'Response': 0.3},
 {'Question': 'At the hardware store, 0.2 the nails are size 2d and 0.5 the nails are size 4d. What fraction of the nails are either size 2d or 4d?',
  'Rationale': 'At the hardware store, 0.2 the nails are size 2d and 

In [31]:
from langchain.prompts import (
    FewShotPromptTemplate,
    PromptTemplate
)

examples = random.sample(list(cot_df.T.to_dict().values()), 10)


template = """
Question: {Question}
Rationale: {Rationale}
Response: {Response}
"""

example_prompt = PromptTemplate(
    input_variables=["Question", "Rationale", "Response"], 
    template=template
)

system_prompt = """
You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.
"""

COT_prompt = FewShotPromptTemplate(
    prefix=system_prompt,
    example_prompt=example_prompt,
    examples=examples,
    suffix="Question: {input}", 
    input_variables=['input']
)


print(COT_prompt.format(input='[EXAMPLE QUESTION]'))


You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?
Rationale: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?\n0.63 - 0.33 = 0.3
<p>
</details>
Response: 0.3



Question: A chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans. How many kilograms of nuts did the chef buy in all?
Rationale: The chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans, so in total they bought 0.12 + 0.38 = 0.50 kilograms of nuts.
Response: 0.5



Question: There are 42 walnut trees and 12 orange trees cur

In [32]:
chain = LLMChain(
    llm=chat_model,
    prompt=COT_prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?
Rationale: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?\n0.63 - 0.33 = 0.3
<p>
</details>
Response: 0.3



Question: A chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans. How many kilograms of nuts did the chef buy in all?
Rationale: The chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans, so in total they bought 0.12 + 0.38 = 0.50 kilograms of nuts.
Response: 0.5




'Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. To find out how much more cinnamon she used, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1 tablespoons.\nResponse: 0.1'

# Self-consistency

In [33]:
chat_model = ChatOpenAI(temperature=0.7)

cot_chain = LLMChain(
    llm=chat_model,
    prompt=COT_prompt,
    verbose=True
)


template = """
Using the following answers, return the answer that comes up more often

{answers}

ANSWER:"""

consistency_prompt = PromptTemplate.from_template(template)

consistency_chain = LLMChain(
    llm=chat_model,
    prompt=consistency_prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

responses = []

for i in range(10):
    response = cot_chain.run(question)
    responses.append(response)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?
Rationale: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?\n0.63 - 0.33 = 0.3
<p>
</details>
Response: 0.3



Question: A chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans. How many kilograms of nuts did the chef buy in all?
Rationale: The chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans, so in total they bought 0.12 + 0.38 = 0.50 kilograms of nuts.
Response: 0.5





> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?
Rationale: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?\n0.63 - 0.33 = 0.3
<p>
</details>
Response: 0.3



Question: A chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans. How many kilograms of nuts did the chef buy in all?
Rationale: The chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans, so in total they bought 0.12 + 0.38 = 0.50 kilograms of nut


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?
Rationale: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?\n0.63 - 0.33 = 0.3
<p>
</details>
Response: 0.3



Question: A chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans. How many kilograms of nuts did the chef buy in all?
Rationale: The chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans, so in total they bought 0.12 + 0.38 = 0.50 kilograms of nut


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Question: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?
Rationale: At a pie-eating contest, Erik got through 0.63 piebefore time was called ; Frank finished just 0.33 pie. How much more pie did Erik eat than Frank?\n0.63 - 0.33 = 0.3
<p>
</details>
Response: 0.3



Question: A chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans. How many kilograms of nuts did the chef buy in all?
Rationale: The chef bought 0.12 kilograms of almonds and 0.38 kilograms of pecans, so in total they bought 0.12 + 0.38 = 0.50 kilograms of nut

In [34]:
answers = '\n\n'.join(responses) 
consistency_chain.run(answers)



> Entering new LLMChain chain...
Prompt after formatting:

Using the following answers, return the answer that comes up more often

Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. To find the difference, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1. So, Mandy used 0.1 tablespoon more cinnamon than nutmeg.
Response: 0.1

Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. To find out how much more cinnamon she used, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1. Therefore, Mandy used 0.1 tablespoon more cinnamon than nutmeg.
Response: 0.1

Rationale: Mandy used 0.6 tablespoons of cinnamon and 0.5 tablespoons of nutmeg. 
To find out how much more cinnamon she used than nutmeg, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1 tablespoons.
Response: 0.1

Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. To fin

'0.1'

# Inception

In [37]:
system_prompt = """
You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.
"""

template = """

{input}

Let's think about it step by step:
"""

prompt = PromptTemplate.from_template(system_prompt + template)

chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

question = """
Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question. 
Your task is to compute the solution using the given arithmetic operations. 
The only arithmetic operators needed to answer the questions are'+'(addition) and'-'(subtraction). 
The answer should be correct to one decimal place.



Mandy made an apple pie. She used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. 
How much more cinnamon than nutmeg did Mandy use?


Let's think about it step by step:


> Finished chain.


'First, we need to calculate the difference between the amounts of cinnamon and nutmeg. To do this, we subtract the amount of nutmeg from the amount of cinnamon.\n\n0.6 tablespoons - 0.5 tablespoons = 0.1 tablespoons\n\nTherefore, Mandy used 0.1 tablespoons more cinnamon than nutmeg.'

# Self-Ask

In [47]:
%pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [69]:
from langchain.agents import (
    initialize_agent,
    AgentType,
    load_tools
)

tools = load_tools(["serpapi"])

tools[0].name = 'Intermediate Answer'

self_ask_with_search = initialize_agent(
    tools=tools, 
    llm=chat_model, 
    agent=AgentType.SELF_ASK_WITH_SEARCH, 
    verbose=True
)

print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [103]:
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

question = "What is the hometown of the reigning men's U.S. Open champion?"

self_ask_with_search.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mot

'Belleville, Illinois'

# ReAct

In [117]:

tools = load_tools(["serpapi", "llm-math"], llm=chat_model)
agent = initialize_agent(
    tools=tools, 
    llm=chat_model, 
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, 
    verbose=True
)

handler = StdOutCallbackHandler()

question = """
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?
"""

agent.run(
    question,
    callbacks=[handler]
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?

Thought:

> Finished chain.
I need to find out who Leo DiCaprio's girlfriend i

'Her current age raised to the 0.43 power is approximately 3.55.'

In [118]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


# Plan and execute

In [119]:
%pip install langchain-experimental

Note: you may need to restart the kernel to use updated packages.


In [136]:
from langchain_experimental.plan_and_execute import (
    PlanAndExecute, 
    load_agent_executor, 
    load_chat_planner
)

chat_model = ChatOpenAI(model_name='gpt-4')

tools = load_tools(["serpapi", "llm-math"], llm=chat_model)

planner = load_chat_planner(chat_model)

executor = load_agent_executor(
    chat_model, 
    tools, 
    verbose=True
)
agent = PlanAndExecute(
    planner=planner, 
    executor=executor, 
    verbose=True
)

In [121]:
print(planner.llm_chain.prompt.format(input='Some input'))

System: Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'
Human: Some input


In [122]:
print(executor.chain.agent.llm_chain.prompt.format(
    previous_steps='previous_steps', 
    current_step='current_step', 
    agent_scratchpad='agent_scratchpad'
))

System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {{'tool_input': {{'type': 'string'}}}}
Calculator: Useful for when you need to answer questions about math., args: {{'tool_input': {{'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to 

In [137]:
question ="""
Who is Leo DiCaprio's girlfriend? 
What is her current age raised to the 0.43 power?
"""
agent.run(question)



> Entering new PlanAndExecute chain...
steps=[Step(value='Research who is the current girlfriend of Leo DiCaprio.'), Step(value='After identifying her, find out her current age.'), Step(value='Calculate her age raised to the power of 0.43 using a calculator or programming language.'), Step(value='Given the above steps taken, please respond to the users original question.\n')]

> Entering new AgentExecutor chain...
Thought: To find out who is the current girlfriend of Leonardo DiCaprio, I can use the search tool to look up recent information on this topic.
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio current girlfriend 2022"
}
```
Observation: Victoria Lamas
Thought:The search results indicate that Victoria Lamas is currently dating Leonardo DiCaprio. I will confirm this in my final response. 
Action:
```
{
  "action": "Final Answer",
  "action_input": "The current girlfriend of Leonardo DiCaprio is Victoria Lamas."
}
```

> Finished chain.
*****

Step: Res

"Victoria Lamas' age (24) raised to the power of 0.43 is approximately 3.92."